In [147]:
# This is the children's story "Hoppelpoppel - wo bist du" by my most favourite author from the last years, Hans Fallada
# as a preparation for a visualization in Tableau.

import pandas as pd
import numpy as np
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from itertools import chain
import re

In [148]:
# Scraping story text

my_url = 'https://www.projekt-gutenberg.org/fallada/hoppelpo/chap001.html'

uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

page_soup = soup(page_html, 'html.parser')

In [149]:
container = page_soup.find_all('p')
container = container[0]

In [150]:
title = container.find_all('h3')
title = str(title[0])

In [151]:
story = container.find_all('p')
paragraphs = []
for i in story:
    paragraphs.append(str(i))

In [152]:
for i in range(len(paragraphs)):
    paragraphs[i] = paragraphs[i].replace('<p>', '')
    paragraphs[i] = paragraphs[i].replace('</p>', '') 

In [153]:
words = []
try:
    for i in range(len(paragraphs)):
        for j in range(len(paragraphs[i])):
            words.append(paragraphs[j].split(' ')) 
except:
    pass

In [154]:
# Flatten nested lists in words

words = list(chain.from_iterable(words))

In [155]:
for i in range(len(words)):
        words[i] = words[i].strip()
        words[i] = words[i].replace('\xa0...', '')
        words[i] = words[i].replace('<a', '')
        words[i] = words[i].replace('</a>', '')
        words[i] = words[i].replace('\n', '')
        words[i] = words[i].replace('-', '')
        words[i] = words[i].replace('–', '')
        words[i] = words[i].replace('.', '')
        words[i] = words[i].replace(':', '')
        words[i] = words[i].replace(',', '')
        words[i] = words[i].replace('>', '')
        words[i] = words[i].replace('<', '')
        words[i] = words[i].replace('›', '')
        words[i] = words[i].replace('‹', '')
        words[i] = words[i].replace('?', '')
        words[i] = words[i].replace('!', '')
        words[i] = words[i].replace('«', '')
        words[i] = words[i].replace('»', '')
        words[i] = words[i].replace('(', '')
        words[i] = words[i].replace(')', '')
        words[i] = words[i].replace(' ', '')
        words[i] = re.sub(str('id='+'"'+'page\d'+'"'), '', words[i])
        words[i] = re.sub(str('name='+'"'+'page\d'+'"'), '', words[i])

In [156]:
# Remove empty strings in words

while("" in words) :
    words.remove("")

In [157]:
df = pd.DataFrame(words)

In [158]:
df.reset_index(inplace = True)

In [159]:
#df.columns = ['rank', 'text']
df.columns = ['rank', 'text']
df['text2'] = ''

In [160]:
# Cleaning last HTML code

part1 = df[df.text == 'title="Dieter7/lac"']
part2 = df[df.text == 'title="suru/Dieter7"']
part3 = df[df.text == 'title="lac/Dieter7"']
eraser = pd.concat([part1, part2, part3])

erank = []
for i in range(len(eraser)):
    erank.append(int(eraser.index[i]))

for i in erank:
    df = df.drop([i])

In [161]:
df.reset_index(drop = True, inplace = True)

In [162]:
df = df.drop('rank', axis = 1)

In [163]:
df.reset_index(inplace = True)

In [164]:
df.columns = ['rank', 'text', 'text2']

In [165]:
# Fill 'text2' with text from previous row

for i in range(len(df)-1):
    if i < len(df):
        try:
            df['text2'][i] = df.text[i+1] 
        except:
            pass

<ipython-input-165-d31fb755fdcd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text2'][i] = df.text[i+1]


In [166]:
df['text3_lowercase'] = df.text.str.lower()

In [167]:
# Setting for displaying the full dataframe 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [168]:
# Creating a dataframe with German words in lowercase for count of unique values

df2 = pd.Series(df.text3_lowercase.unique())
df2 = pd.DataFrame(df2)

In [169]:
df2.columns = ['text_unique']
df2['count_text'] = ''

In [170]:
# Counting the unique words

for i in range(len(df2)):
    x = df2.text_unique[i]      # x stores each single word in text_unique
    df2['count_text'][i] = df[df.text3_lowercase == x].text3_lowercase.count()

In [171]:
# df['count_text'] = ''
# df['lat'] = ''
# df['lon'] = ''
df['path_ID'] = ''
#df['path_start_end'] = ''

In [172]:
# for i in range(len(df2)):
#     x = df2.text_unique[i]
#     for j in range(len(df)):
#         if df.text3_lowercase[j] == x:
#             df['count_text'][j] = df2.count_text[i] 

In [173]:
df3 = pd.read_csv('coordinates.csv', sep = ';')

In [174]:
df3 = df3.drop(['no', 'angle', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'], axis = 1)

In [175]:
df2 = pd.concat([df2, df3], axis = 1)

In [176]:
df.head(5)

,rank,text,text2,text3_lowercase,path_ID
0,0,Es,war,es,
1,1,war,einmal,war,
2,2,einmal,ein,einmal,
3,3,ein,kleiner,ein,
4,4,kleiner,Junge,kleiner,


In [177]:
##########################
# path_ID ist falsch zusammengesetzt! 
# Für die tatsächlichen Pfade von einem Wort zu mehreren Folgeworten muss der rank des unique_texts genommen werden!
########################## 

for i in range(len(df)):
    if i < len(df)-1:
        df.path_ID[i] = str(df['rank'].iloc[i]) + '_' + str(df['rank'].iloc[i+1])
    else:
        df.path_ID[i] = str(df['rank'].iloc[-1]) + '_' + str(df['rank'].iloc[-1])

<ipython-input-177-1aa7e2125a67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.path_ID[i] = str(df['rank'].iloc[i]) + '_' + str(df['rank'].iloc[i+1])
<ipython-input-177-1aa7e2125a67>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.path_ID[i] = str(df['rank'].iloc[-1]) + '_' + str(df['rank'].iloc[-1])


In [178]:
df4 = df.copy()

In [179]:
df['path_start_end'] = 'start'
df4['path_start_end'] = 'end'

In [180]:
df = pd.concat([df, df4])

In [181]:
df.reset_index(drop = True, inplace = True)

In [182]:
df.tail()

,rank,text,text2,text3_lowercase,path_ID,path_start_end
2593,1294,die,Welt,die,1294_1295,end
2594,1295,Welt,wieder,welt,1295_1296,end
2595,1296,wieder,in,wieder,1296_1297,end
2596,1297,in,Ordnung,in,1297_1298,end
2597,1298,Ordnung,,ordnung,1298_1298,end


In [183]:
df.to_csv('fallada_hoppelpoppel.csv')

In [184]:
df2.to_csv('hoppelpoppel_coordinates.csv')